In [1]:
import os
import numpy as np
import pandas as pd
import notebooks_loops_script as nls
import backend_codes.load_subsets as ls

# Origin Destination (OD) Matrices
This notebook creates OD matrices form the respective rolling windows.

In [2]:
# Rolling Window Parameters
start = "20200406"
end = "20200409"
denom = "3days"
allow_even_subsets = False
sam = False
del_one_tweeters = True

In [3]:
middle = nls.middeling(start, end, allow_even_subsets=allow_even_subsets)

In [4]:
mm_path = f'./data/movement_matrices_notebookdemo/{denom}/mm_{middle}.npy'
if type(sam) == int:
    mm_path = mm_path.split('.npy')[0] + f'_{str(sam)}.npy'

In [5]:
path = mm_path.split("/mm")[0]
if not os.path.exists(path):
    os.mkdir(path)

# Creating OD Matrices 
We now use the user_dicts to get some infos about the users tweeting habits. A function is desinged, the constructs an Origin-Destination Matrix from the respective rolling window.

In [6]:
tweets_df = ls.load_and_subset(start, end, del_one_tweeters=del_one_tweeters, samp_size=sam, tweets_path="data/tweets/preprocessed_tweets_with_poi_location.csv")

In [7]:
def create_matrix(df, diag_to_0=False):   
    # reset index
    df = df.reset_index(drop=True)
    
    # create list of the unique user IDs
    user_ids = df.User_ID.unique().tolist()

    # create a dict, with an empty list for ea user ID
    user_sequence = {_id: [] for _id in user_ids}

    # fill the dict with all visited Barrio IDs (they are sorted by timestamp for each user)
    for i in range(len(df)):
        user_sequence[df.User_ID[i]].append(df.cod[i])

    # create the full movement matrix
    matrix = np.zeros((164, 164))
    for key in user_sequence:
        _len = len(user_sequence[key])
        if _len > 0:
            for i in range(0, _len-1):
                matrix[user_sequence[key][i], user_sequence[key][i+1]] = \
                    matrix.item((user_sequence[key][i], user_sequence[key][i+1])) + 1

    
    # set diagonal to 0
    def fun_diag_to_0(matrix):
        for i in range(0, 164):
            matrix[i, i] = 0
        return matrix
    
    
    if diag_to_0:
        matrix = fun_diag_to_0(matrix)
        

    full_movement_matrix = pd.DataFrame(matrix)
    full_movement_matrix = full_movement_matrix.drop([0]).drop([0], axis=1) # drop first row and column

    
    return full_movement_matrix, user_sequence

In [8]:
full_movement_matrix, user_sequence = create_matrix(tweets_df, diag_to_0=True)
np.save(mm_path, full_movement_matrix.to_numpy())